In [1]:
import numpy as np

Modèle de l'ABC Gibbs

In [5]:

def ABC_Gibbs(theta0, x_etoile, epsilon, s_j, N):

     """
    
    Arguments :
    x_etoile : données observées
    N : nombre d'itérations
    quantile : quantile pour le seuil ε
    summary_statistic : statistique résumée s
    
    Renvoie un échantillon (θ^(1),..., θ^(N)).
    """
     
    # Initialiser la liste pour stocker les échantillons
    échantillons = []

    # Ajouter le point de départ comme le premier échantillon
    échantillons.append(theta0)

    # Nombre de paramètres
    n = len(theta0)

    for i in range(1, N):
        # Créer un nouvel échantillon à partir de l'échantillon précédent
        nouvel_échantillon = échantillons[-1].copy()

        for j in range(n):
            # Vérifier si εj est égal à zéro et si sj est une statistique conditionnellement suffisante
            if epsilon[j] == 0 and sj_is_sufficient:
                # Si oui, effectuer une simulation exacte à partir de la distribution conditionnelle correspondante
                # (implémentation de cette partie dépend de sj_is_sufficient)
                nouvel_échantillon[j] = exact_simulation_from_conditional(...)
            else:
                # Sinon, simuler à partir de la distribution conditionnelle approximative
                # en utilisant une méthode de Monte Carlo par exemple
                # Ici, nous prenons εj comme un quantile de distance empirique
                epsilon_j_quantile = np.percentile(epsilon[j], 90)  # Par exemple, le 90e percentile
                nouvel_échantillon[j] = simulate_from_approx_conditional(x_etoile, nouvel_échantillon, epsilon_j_quantile, s_j, j)

        # Ajouter le nouvel échantillon à la liste des échantillons
        échantillons.append(nouvel_échantillon)

    return échantillons

# Fonction pour simuler à partir de la distribution conditionnelle approximative
def simulate_from_approx_conditional(observations, previous_sample, epsilon_j_quantile, sj, j):
    # Implémenter la simulation à partir de la distribution conditionnelle approximative
    # en utilisant une méthode comme la méthode de Monte Carlo par exemple
    ...

# Fonction pour simuler une distribution conditionnelle exacte
def exact_simulation_from_conditional(...):
    # Implémenter la simulation exacte à partir de la distribution conditionnelle
    ...

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 15)

Example d'utilisation avec le toy Normal–Normal model

σ = 1, K = 10, n = 20

In [6]:
import time

# Capture du temps au début de l'exécution
start_time = time.process_time()

# Définition des paramètres du modèle : sigma_carré et chi_carré
sigma_carré = 1
chi_carré = 1

# Fonction pour simuler les données
def simuler_données(alpha, n, K):
    mu = np.random.normal(loc=alpha, scale=np.sqrt(chi_carré), size=n)
    observations = np.random.normal(loc=mu[:, np.newaxis], scale=np.sqrt(sigma_carré), size=(n, K))
    return observations

# Fonction pour calculer la statistique résumée (moyenne)
def statistique_résumé(données, alpha):
    return np.mean(données - alpha)

# Fonction pour simuler à partir de la distribution conditionnelle approximative
def simuler_depuis_loi_cond_approx(observations, previous_sample, epsilon_j_quantile, sj, j):
    # Générer des échantillons de Alpha à partir de la distribution uniforme
    alpha_échantillons = np.random.uniform(low=-4, high=4, size=1000)  # Utiliser 1000 échantillons pour l'illustration

    # Calculer la statistique résumée (moyenne) pour chaque échantillon de Alpha
    statistique_résumée = np.array([sj(observations, alpha) for alpha in alpha_échantillons])

    # Calculer les distances entre la statistique résumée observée et celle calculée à partir des échantillons de Alpha
    distances = np.abs(statistique_résumée - sj(observations, previous_sample[j]))

    # Sélectionner les échantillons de Alpha associés aux plus petites distances (quantile de distance)
    échantillons_sélectionnés = alpha_échantillons[distances < epsilon_j_quantile]

    # Prendre un échantillon aléatoire parmi les échantillons sélectionnés
    if len(échantillons_sélectionnés) > 0:
        new_alpha_échantillon = np.random.choice(échantillons_sélectionnés)
    else:
        # S'il n'y a pas d'échantillon sélectionné, utiliser le dernier échantillon comme valeur par défaut
        new_alpha_échantillon = previous_sample[j]

    return new_alpha_échantillon

# Fonction pour simuler une distribution conditionnelle exacte
def simulation_loi_cond_exacte(x_star):
    # La distribution conditionnelle d'Alpha sachant les données observées suit une distribution uniforme
    alpha_échantillon = np.random.uniform(low=-4, high=4)
    return alpha_échantillon



In [7]:
# Par exemple, le 90e quantile
quantile = 90

def ABC_Gibbs(theta0, x_etoile, epsilon, s_j, N):
    # Initialiser la liste pour stocker les échantillons
    échantillons = []

    # Ajouter le point de départ comme le premier échantillon
    échantillons.append(theta0)

    # Nombre de paramètres
    n = len(theta0)

    for i in range(1, N):
        # Créer un nouvel échantillon à partir de l'échantillon précédent
        nouvel_échantillon = échantillons[-1].copy()

        for j in range(n):
            # Simuler à partir de la distribution conditionnelle approximative
            epsilon_j_quantile = np.percentile(epsilon[j], quantile)  
            nouvel_échantillon[j] = simuler_depuis_loi_cond_approx(x_etoile, nouvel_échantillon, epsilon_j_quantile, s_j, j)

        # Ajouter le nouvel échantillon à la liste des échantillons
        échantillons.append(nouvel_échantillon)

    return échantillons

# Définir les paramètres initiaux et les observations
theta0 = np.random.uniform(low=-4, high=4, size=20)  # Hypothèse initiale pour alpha
n = 20  # Nombre de moyennes à estimer
K = 10  # Nombre d'observations par moyenne
x_etoile = simuler_données(theta0, n, K)  # Données observées

# Définir les paramètres de l'algorithme ABC-Gibbs
epsilon = [0.1] * n  # Tolerance pour chaque paramètre
N = 1000  # Nombre d'itérations

# Exécuter l'algorithme ABC-Gibbs
result_samples = ABC_Gibbs(theta0, x_etoile, epsilon, statistique_résumé, N)

# Afficher les échantillons résultants
print("Échantillons résultants:")
for sample in result_samples:
    print(sample)

# Capture du temps à la fin de l'exécution
end_time = time.process_time()

# Calcul du temps écoulé
cpu_time = end_time - start_time

# Affichage du temps CPU
print("Temps CPU:", cpu_time, "secondes")

Échantillons résultants:
[-0.2318621   3.16591413 -1.66178945 -2.54160589 -0.78454451  0.32797156
 -3.59864937  1.65492157  2.50116889 -3.15689937 -1.62670238  0.29996838
  1.51774751  1.24810158 -3.39701745  2.46611756 -3.11900806 -1.64074957
 -3.86012572  3.74829979]
[-0.30134889  3.12427181 -1.7001684  -2.62746245 -0.70028625  0.36493426
 -3.59872504  1.64488875  2.45145996 -3.05709122 -1.65813442  0.24435341
  1.56107406  1.15429499 -3.39056614  2.45125073 -3.04115003 -1.54598073
 -3.92008554  3.82090755]
[-0.39583268  3.10570076 -1.76716552 -2.53302556 -0.76874965  0.44410916
 -3.67244541  1.7413725   2.46161309 -3.1083355  -1.56151986  0.25800497
  1.57330425  1.17597689 -3.39815134  2.42849734 -3.00511721 -1.51867127
 -3.9191283   3.78238299]
[-0.30056272  3.08779097 -1.79326555 -2.55114769 -0.68249273  0.46871823
 -3.67431249  1.76051179  2.44182494 -3.06396356 -1.59208222  0.24172906
  1.49193273  1.10359497 -3.41052843  2.49818017 -3.00281997 -1.56061906
 -3.95997324  3.69888

Erreur d'inférence

In [8]:
def inferential_error(samples, observations):
    # Calculer la moyenne des données observées pour chaque moyenne
    observations_mean = np.mean(observations, axis=1)

    # Calculer la distance euclidienne entre les échantillons et les moyennes des données observées
    errors = [np.linalg.norm(sample - observations_mean) for sample in samples]

    # Calculer la moyenne des erreurs
    mean_error = np.mean(errors)
    
    return mean_error

# Calculer l'erreur d'inférence pour les échantillons résultants
error = inferential_error(result_samples, x_etoile)
print("Erreur d'inférence:", error)

Erreur d'inférence: 7.698014846994055


Erreur de Monte Carlo

In [9]:
def monte_carlo_error(samples):
    # Calculer l'écart type des échantillons
    std_dev = np.std(samples)

    # Nombre total d'échantillons
    num_samples = len(samples)

    # Calculer l'erreur de Monte Carlo
    mc_error = std_dev / np.sqrt(num_samples)
    
    return mc_error

# Calculer l'erreur de Monte Carlo pour les échantillons résultants
mc_error = monte_carlo_error(result_samples)
print("Erreur de Monte Carlo:", mc_error)

Erreur de Monte Carlo: 0.07377098147241559
